In [ ]:
import re
def extract_from_string(string,mode='[]'):
    letters = [char for char in string if char in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ']
    return letters

In [ ]:
data_name = 'rank_zephyr_IR'
model_name = 'Llama-3.2-3B-Instruct' #must be it 
num_candidate  = 20
new_tokens = ['%s'%chr(ord('A')+i) for i in range(num_candidate)]


In [ ]:
import re

def generate_content(text, num_candidate):
    text = text.split('\n')
    content = []

    # 构造替换字符串
    pattern1 = r'(with\s+)\d+(\s+passages)'
    repl1 = lambda m: f"{m.group(1)}{num_candidate}{m.group(2)}"
    content.append(re.sub(pattern1, repl1, text[0]))

    content.append(text[1])

    # 取中间 passage
    content.extend(text[2:-2][:num_candidate])

    content.append(text[-2])

    pattern2 = r'(the\s+)\d+(\s+passages)'
    repl2 = lambda m: f"{m.group(1)}{num_candidate}{m.group(2)}"
    content.append(re.sub(pattern2, repl2, text[-1]))

    return '\n'.join(content)


In [ ]:
#数据读取
import sys
import os
import torch
from datasets import Dataset,load_dataset,load_from_disk


# 获取上一级目录的路径
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, parent_dir)
os.environ["CUDA_VISIBLE_DEVICES"] = "2"  # 只使用 GPU 3
if data_name == 'rank_zephyr_IR':
    ds = load_dataset("rryisthebest/rank_zephyr_training_data_alpha")
    train_data = ds['train']
if data_name == 'quora':
    train_data = load_from_disk("quora_original/quora")
if data_name == 'Games':
    train_data = load_from_disk("Games_original/Games")

train_list = train_data.to_list()
for index in range(len(train_list)):
    prompt =  train_list[index]['conversations'][1]['value']
    prompt = prompt.replace('The output format should be [] > [], e.g., [B] > [A]','')
    prompt = prompt.replace('each indicated by a alphabetic identifier []','each indicated by an identifier infront of the sentence')
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        prompt = prompt.replace("[%s]"%letter, f"{letter}") 
    prompt += """\nNotice: 1. Return a single line with the identifiers in descending order, separated by "|" (e.g.,STRICTLY OUTPUT FORMAT: |C|B|A|D|); 2. None space between the identifiers. 3. We have %s candidates in total, so the output should be a ranking of %s identifiers."""%(num_candidate,num_candidate)
        
    train_list[index]['conversations'][1]['value'] = generate_content(prompt,num_candidate)
train_data = Dataset.from_list(train_list)
#去除特别少的<
def func_1(example): # 去除有重复元素
    extracted = extract_from_string(example['conversations'][2]['value'])
    return len(extracted) >= 20#保留ranking大于10的部分   
train_data = train_data.filter(func_1)


train_list = train_data.to_list()


In [ ]:
print(train_data[0]['conversations'][1]['value'])

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM
# 加载 Tokenizer
model_path = "/data/zoo/%s"%model_name
def initialize_models(model_path):
    """Initialize base model and tokenizer"""
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    
    llm = AutoModelForCausalLM.from_pretrained(
        model_path,
        device_map='cuda',
        torch_dtype=torch.bfloat16,
        attn_implementation='flash_attention_2',
    )
    # add new tokenizer
    # old_tokens = [chr(ord('A')+i) for i in range(num_candidate)]
    # tokenizer.add_tokens(new_tokens)
    # llm.resize_token_embeddings(len(tokenizer))
    # llm.eval()
    # with torch.no_grad():
    #     for i in range(num_candidate):
    #         llm.lm_head.weight[tokenizer.encode('|%s|'%chr(ord('A')+i),add_special_tokens=False)] = llm.lm_head.weight[tokenizer.encode(chr(ord('A')+i),add_special_tokens=False)]
    #         llm.model.embed_tokens.weight[tokenizer.encode('|%s|'%chr(ord('A')+i),add_special_tokens=False)] = llm.model.embed_tokens.weight[tokenizer.encode(chr(ord('A')+i),add_special_tokens=False)]
    return llm, tokenizer
llm,tokenizer = initialize_models(model_path)

In [ ]:
# dataset 加载
from utils import GenerationDataset, combined_collate_fn, sample_top_p
from torch.utils.data import Dataset, DataLoader, Subset



all_dataset = GenerationDataset(train_data, tokenizer, combined=True)
all_dataloader = DataLoader(all_dataset, batch_size=1)
#               






In [ ]:
len(all_dataloader)

In [ ]:
if 'Llama' in model_name:
    end_token = '<|eot_id|>'
if 'Qwen' in model_name:
    end_token = '<|im_end|>'

def create_logits_processor_llama(corpus,start):
    corpus_id = set([tokenizer.convert_tokens_to_ids(letter) for letter in corpus]) #| 
    token_id4 = tokenizer.convert_tokens_to_ids(end_token)
    token_id1 = tokenizer.convert_tokens_to_ids('|')
    def process_token(token_ids, logits):
        token_ids = token_ids[0][start:]
        # print(token_ids)
        if len(token_ids)%2 == 1:
            logits[:,list(corpus_id)] += 50
        else:
            logits[:,token_id1] += 100
        token_ids_set = set(token_ids.tolist())
        exist_token = list(set(token_ids_set)&corpus_id)
        # print(corpus_id)
        # if not
        logits[:,exist_token] = - 10000000
        # print(token_ids)
        # print(exist_token)
        if len(exist_token) == len(corpus_id): #不把corpus_id中的token全部生成,就不能结束
            logits[:,token_id4] = 1000
        return logits
    return process_token

In [ ]:
def generate_text_self_regression_with_constraint(model,tokenizer,batch):
    text,GT_label,ranking = batch
    input_encoding = tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                truncation=True,
            )
    # 获取input_ids
    input_ids = input_encoding['input_ids'].cuda()
    #指定生成的前几个token
    # add_token = torch.tensor(tokenizer.encode('|')[1:])
    #合并
    # input_ids = torch.cat([input_ids, add_token.unsqueeze(0)],dim=1).cuda()     #指定生成的前几个token

    input_length = input_ids.shape[1]

    corpus = new_tokens
    # print(corpus)
    process_token = create_logits_processor_llama(corpus,input_length)

    # 4. 生成文本
    with torch.no_grad():  # 关闭梯度计算，减少显存占用
        output_ids = model.generate(
            input_ids=input_ids, 
            max_length=5000,                   # 与 sampling_params 的 max_tokens 对齐
            do_sample=False,                   # temperature=0.0 时一般设置为False
            temperature=0.0,                   # 完全确定性生成
            pad_token_id=tokenizer.eos_token_id,
            logits_processor=[process_token]   # 与 sampling_params 中的 logits_processors 对齐
        )


    # 5. 解码输出
    generated_ids = output_ids[:, input_length:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
def generate_text_self_regression(model,tokenizer,batch):
    text,GT_label,ranking = batch
    input_encoding = tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                truncation=True,
            )
    # 获取input_ids
    input_ids = input_encoding['input_ids'].cuda()
    #指定生成的前几个token
    # add_token = torch.tensor(tokenizer.encode('|')[1:])
    #合并
    # input_ids = torch.cat([input_ids, add_token.unsqueeze(0)],dim=1).cuda()     #指定生成的前几个token

    input_length = input_ids.shape[1]

    corpus = new_tokens
    # print(corpus)
    process_token = create_logits_processor_llama(corpus,input_length)

    # 4. 生成文本
    with torch.no_grad():  # 关闭梯度计算，减少显存占用
        output_ids = model.generate(
            input_ids=input_ids, 
            max_length=5000,                   # 与 sampling_params 的 max_tokens 对齐
            do_sample=False,                   # temperature=0.0 时一般设置为False
            temperature=0.0,                   # 完全确定性生成
            pad_token_id=tokenizer.eos_token_id,
            # logits_processor=[process_token]   # 与 sampling_params 中的 logits_processors 对齐
        )


    # 5. 解码输出
    generated_ids = output_ids[:, input_length:]
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

In [ ]:
from tqdm import tqdm
import re
break_flag = False
wrong_cache = []
index = -1
for batch in tqdm(all_dataloader):
    index += 1
    GT_label = batch[1][0][: batch[1][0].rfind(']') + 1]
    # generate_label1 = generate_text_self_regression(llm,tokenizer,batch)
    generate_label = generate_text_self_regression_with_constraint(llm,tokenizer,batch)
    generate_label_list = extract_from_string(generate_label,"|")
    # print(GT_label)
    # print(generate_label)
    if set(generate_label_list) == set([chr(ord('A')+i) for i in range(num_candidate)]):
        train_list[index]['conversations'][-1]['value'] = generate_label
        # print(generate_label_list)
    else:
        print("Also unexpected error in revised generated label")
        # print(generate_label)       

    
    # break

In [ ]:
from datasets import Dataset, DatasetDict
save_data = Dataset.from_list(train_list)#.shuffle()
save_data.save_to_disk("./%s_%s/%s"%(data_name,num_candidate,model_name))

In [ ]:
# from datasets import load_from_disk

# data = load_from_disk("./rank_zephyr_IR_10/Llama-3.2-3B-Instruct")

In [ ]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

In [ ]:
# from datasets import load_from_disk

# def func_1(example): # 去除有重复元素
#     extracted = extract_from_string(example['conversations'][2]['value'],'|')
#     c1 = len(extracted) < 20
#     c2 =  len(extracted) > 10
#     return c1 and c2 #保留ranking大于10的部分   
# ds = load_from_disk('/data/yingpeng/efficient_inference/dataset/rank_zephyr_IR_copy/Llama-3.2-3B-Instruct')
# ds = ds.filter(func_1)

# # Shuffle the dataset (set seed for reproducibility if desired)
# ds_shuffled = ds.shuffle(seed=42)

# # Save the shuffled dataset to disk
# ds_shuffled.save_to_disk('/data/yingpeng/efficient_inference/dataset/rank_zephyr_IR_10/Llama-3.2-3B-Instruct')


In [ ]:
len(ds_shuffled)

In [ ]:
>H>F>C>I>A>J>M>N>P>Q>R>S>T>U>V<W>X>Y>Z
>H>F>C>I>A>J>M>N>P>S>T>R>B>D>L>K>E>O>G>Q
--------------------------------
  0%|          | 2/9978 [00:03<4:02:21,  1.46s/it]
>F>A>I>J>L>O>P>S>T
>F>A>I>J>L>O>P>S>T>R>K>H>C>B>M>D>E>G>N>Q
--------------------------------
  0%|          | 3/9978 [00:04<4:08:18,  1.49s/it]
>C>A>E>H>J>O>N>P>S>T>Q>R>M>L>K>U>V>W>X>Y>Z
>C>A>E>H>J>O>N>P>S>T>R>M>K>L>F>B>D>G>I>Q
--------------------------------
  0%|          | 4/9978 [00:05<3:55:05,  1.41s/it]
>D>A>E>F>G>H>I>J>K>L>M>N>O>P>R>S>T
>D>A>E>F>G>H>I>J>K>L>M>N>O>P>R>S>T>B>C>Q